# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(override=True) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.agents import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [11]:
search.run("sf")

'42f'

In [12]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [13]:
get_current_temperature.name

'get_current_temperature'

In [14]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [15]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [16]:
from langchain.tools.render import format_tool_to_openai_function

In [17]:
format_tool_to_openai_function(get_current_temperature)

/var/folders/wj/kn8jgh51669650nr3m4hlnhc0000gn/T/ipykernel_12804/540789164.py:1: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [18]:
get_current_temperature({"latitude": 13, "longitude": 14})

/var/folders/wj/kn8jgh51669650nr3m4hlnhc0000gn/T/ipykernel_12804/2143878976.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_current_temperature({"latitude": 13, "longitude": 14})


'The current temperature is 19.6°C'

In [19]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [20]:
search_wikipedia.name

'search_wikipedia'

In [21]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [22]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [23]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.\n\n\n\nPage

In [24]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
#from langchain_community.tools import OpenAPISpec
from langchain.utilities.openapi import OpenAPISpec
from openapi_pydantic import OpenAPI, Info, PathItem, Operation, Response

In [25]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [26]:
spec = OpenAPISpec.from_text(text)

In [27]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [28]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [29]:
pet_callables 

<function langchain.chains.openai_functions.openapi.openapi_spec_to_openai_fn.<locals>.default_call_api(name: 'str', fn_args: 'dict', headers: 'Optional[dict]' = None, params: 'Optional[dict]' = None, **kwargs: 'Any') -> 'Any'>

In [30]:
from langchain_openai.chat_models import ChatOpenAI

In [31]:
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",temperature=0).bind(functions=pet_openai_functions)

In [32]:
model.invoke("what are three pets names") 

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 119, 'total_tokens': 135, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-1f553373-9499-4aae-a82f-e5e362562b6d-0', usage_metadata={'input_tokens': 119, 'output_tokens': 16, 'total_tokens': 135, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [33]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 122, 'total_tokens': 142, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-59a7a3f3-a416-464e-8d55-1141a1632124-0', usage_metadata={'input_tokens': 122, 'output_tokens': 20, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [34]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0).bind(functions=functions)

In [35]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 101, 'total_tokens': 127, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-d32a4e7d-69fb-4256-bc45-dbfdb57b58bb-0', usage_metadata={'input_tokens': 101, 'output_tokens': 26, 'total_tokens': 127, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [36]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"LangChain"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 97, 'total_tokens': 113, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-3d7c1daf-ffb9-462d-9eb9-8efec9dda2ae-0', usage_metadata={'input_tokens': 97, 'output_tokens': 16, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [37]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",temperature=0).bind(functions=pet_openai_functions)
chain = prompt | model

In [38]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='I can\'t check the weather for you, but you can easily find it on a weather website or app. Just type "current weather in San Francisco" into your favorite search engine, and you\'ll get the latest updates. Easy peasy!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 130, 'total_tokens': 178, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-192d8597-da92-4483-b155-249797019dab-0', usage_metadata={'input_tokens': 130, 'output_tokens': 48, 'total_tokens': 178, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [39]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [40]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [50]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [51]:
get_current_temperature(result.tool_input)

AttributeError: 'AgentFinish' object has no attribute 'tool_input'

In [52]:
result = chain.invoke({"input": "hi!"})

In [53]:
type(result)

langchain_core.agents.AgentFinish

In [54]:
result.return_values

{'output': "Hey there! What's up? Need some help or just here to chat?"}

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
@tool
def get_list_pets(limit: dict) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.mrwinston.com/api"
    
    # Parameters for the request
    print(f"Given input:{limit}")
    params = {'app_name': 'selforder', 'api_key': 'seTK5pxa8tnJHj93'}

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")
        
    data = results['data']
    
    return f'The current temperature is {data}'

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "listPets": get_list_pets
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [ ]:
model.invoke("what are three pets names")

In [ ]:
result = chain.invoke({"input": "what are three pets names"})

In [44]:
result

AgentFinish(return_values={'output': "I can't check the weather for you, but I can suggest you look it up on a weather website or app. It's probably foggy, though—classic San Francisco!"}, log="I can't check the weather for you, but I can suggest you look it up on a weather website or app. It's probably foggy, though—classic San Francisco!")

In [45]:
result = chain.invoke({"input": "What is langchain?"})

In [46]:
result

AgentFinish(return_values={'output': "LangChain is a framework designed for developing applications powered by language models. It provides a set of tools and components that make it easier to build applications that can understand and generate human-like text. LangChain allows developers to create complex workflows that involve language models, enabling functionalities like chatbots, question-answering systems, and more.\n\nKey features of LangChain include:\n\n1. **Modularity**: It offers various modules for different tasks, such as text generation, summarization, and conversation management.\n2. **Integration**: LangChain can integrate with various language models and APIs, making it flexible for different use cases.\n3. **Chain of Thought**: It supports the concept of chaining together multiple operations, allowing for more sophisticated interactions and processing.\n4. **Memory**: LangChain can maintain context over conversations, which is essential for creating more natural and c

In [47]:
chain.invoke({"input": "hi!"})

AgentFinish(return_values={'output': "Hey there! What's up? Need some help or just here to chat?"}, log="Hey there! What's up? Need some help or just here to chat?")